In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
def one_hot(y):
    num_classes = 100
    num_examples = len(y)
    result = np.zeros((num_examples, num_classes))
    result[np.arange(num_examples), y.flatten()] = 1
    return result

In [6]:
# Instantiate model 
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = False
    
top_model = Sequential()
top_model.add(Flatten(input_shape=conv_base.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(100, activation='softmax'))
for layer in top_model.layers:
    layer.trainable = True 

final_model = Model(inputs=conv_base.input, outputs=top_model(conv_base.output))
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)

datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
epochs = 50
generator = datagen.flow(x_train, y_train_oh, batch_size=batch_size)

history = final_model.fit_generator(generator,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=(x_test, y_test_oh)
                                   )

Epoch 1/10
1562/1562 [==============================] - 15s 10ms/step - loss: 3.1883 - acc: 0.2315 - val_loss: 13.0374 - val_acc: 0.1706
Epoch 2/10
1562/1562 [==============================] - 15s 9ms/step - loss: 2.6958 - acc: 0.3229 - val_loss: 13.1462 - val_acc: 0.1662
Epoch 3/10
1562/1562 [==============================] - 15s 9ms/step - loss: 2.5308 - acc: 0.3508 - val_loss: 13.0734 - val_acc: 0.1715
Epoch 4/10
1562/1562 [==============================] - 15s 9ms/step - loss: 2.4212 - acc: 0.3747 - val_loss: 13.1339 - val_acc: 0.1688
Epoch 5/10
1562/1562 [==============================] - 15s 9ms/step - loss: 2.3287 - acc: 0.3946 - val_loss: 13.1318 - val_acc: 0.1699
Epoch 6/10
1556/1562 [============================>.] - ETA: 0s - loss: 2.2504 - acc: 0.4123